In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

Download the source code for DECORP's webpage.

In [15]:
import requests

# URL of the website to scrape
url = "http://www.decorp.com.ph/gencharge/2022.html"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the content of the response (HTML source code)
    html_content = response.text
    
    # Save the HTML source code to a file
    with open("decorp_rates.html", "w", encoding="utf-8") as file:
        file.write(html_content)
    
    print("HTML source code has been downloaded and saved as 'tarelco2_rates.html'.")
else:
    print(f"Failed to retrieve the website. Status code: {response.status_code}")


HTML source code has been downloaded and saved as 'tarelco2_rates.html'.


### **Processing**

In [85]:
from bs4 import BeautifulSoup

# Read the HTML file
with open('decorp_rates.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'lxml')

# Find the table in the HTML (assuming there's only one table)
table = soup.find('table')

# Initialize a list to store the table data
table_data = []

# Process each row in the table
for row in table.find_all('tr'):
    cells = row.find_all('td')

    # Extract text from cells, split by <br>, and trim whitespace
    cell_texts = [cell.get_text(separator='\n').split('\n') for cell in cells]

    # Extract relevant columns (first column + columns from the fourth onwards)
    if len(cells) >= 4:
        first_column_value = cell_texts[0][0] if cell_texts[0] else None
        relevant_columns_texts = cell_texts[3:]  # Columns from the 4th onward

        # Determine the maximum number of rows needed for this table
        max_rows = max(len(texts) for texts in relevant_columns_texts)
        print(max_rows)
        
        # Create rows and populate with None initially
        for i in range(max_rows):
            if len(table_data) <= i:
                # Initialize the row with None values, plus one for the first column
                table_data.append([None] * (len(relevant_columns_texts) + 1))
            
            table_data[i][0] = first_column_value  # Repeat the first column value
            
            # Populate the relevant columns
            for j, texts in enumerate(relevant_columns_texts): #for each column
                if i < len(texts):
                    table_data[i][j + 1] = texts[i]

# Convert the table data to a DataFrame
df = pd.DataFrame(table_data)

df

8
4
7
13


IndexError: list assignment index out of range